In [38]:
import numpy as np
import pandas as pd
import datetime as dt
from google.cloud import aiplatform

In [45]:
endpoint = aiplatform.Endpoint(
    endpoint_name="projects/119551364473/locations/us-central1/endpoints/6444916049069473792")


In [46]:
solar_data = pd.read_csv('gs://forecast_proj_resources/SolarMonthlyData_2657Custs.csv')

In [47]:

# Data preprocessing
# ======================================================================================
consumption_billrate = ['PK', 'LVP', 'SH', 'LVS','OP','OP1','OP2']
solar_data_consumption = solar_data[(solar_data['Netwk Bill Rate Type'].isin(consumption_billrate)) & (solar_data['Unit of me'] == 'KWH')].copy()
solar_data_consumption['Consumption Month'] = solar_data_consumption['Consumption Month'].apply(lambda x: "{:.4f}".format(x))
solar_data_consumption['Consumption Month'] = solar_data_consumption['Consumption Month'].astype('str')
solar_data_consumption['Consumption Month'] = pd.to_datetime(solar_data_consumption['Consumption Month'].apply(lambda x: dt.datetime.strptime(x, '%m.%Y')))
solar_data_consumption['house_type'] = 'solar'

generation_billrate = ['PGR', 'SGR','OGR','OGG','PGG','SGG']
solar_data_generation = solar_data[(solar_data['Netwk Bill Rate Type'].isin(generation_billrate)) & (solar_data['Unit of me'] == 'KWH')].copy()
solar_data_generation['Consumption Month'] = solar_data_generation['Consumption Month'].apply(lambda x: "{:.4f}".format(x))
solar_data_generation['Consumption Month'] = solar_data_generation['Consumption Month'].astype('str')
solar_data_generation['Consumption Month'] = pd.to_datetime(solar_data_generation['Consumption Month'].apply(lambda x: dt.datetime.strptime(x, '%m.%Y')))


solar_grouped_generation  = solar_data_generation.groupby(['Customer ID', 'Consumption Month']).agg({'Sum': 'sum'}).reset_index()
solar_grouped_consumption = solar_data_consumption.groupby(['Customer ID', 'Consumption Month']).agg({'Sum': 'sum'}).reset_index()

solar_net_consumption = solar_grouped_consumption.merge(solar_grouped_generation, on = [ 'Customer ID', 'Consumption Month'], how = 'left',
          suffixes=('_left', '_right'))

solar_net_consumption['Consumption'] = solar_net_consumption.fillna(0)['Sum_left'] - solar_net_consumption.fillna(0)['Sum_right']

solar_net_consumption.drop(['Sum_left','Sum_right'], axis=1, inplace=True)

df = solar_net_consumption.copy()

# Feature Engineering 
def create_lag_features(data, lag_steps):
    for lag in range(1, lag_steps + 1):
        data[f'lag_{lag}'] = data.groupby('Customer ID')['Consumption'].shift(lag)
    return data
df_long = create_lag_features(df, lag_steps=3)

def create_rolling_mean_features(data, window_size):
    data['rolling_mean'] = data.groupby('Customer ID')['Consumption'].transform(lambda x: x.shift(1).rolling(window=window_size).mean())
    return data
df_long = create_rolling_mean_features(df_long, window_size=2)


df_long.fillna(method='bfill', inplace=True)
df_long['Consumption Month'] = pd.to_datetime(df_long['Consumption Month'])



# splittting the data 
split_date = '2014-12-01'
train_data = df_long[df_long['Consumption Month'] < split_date]
test_data = df_long[df_long['Consumption Month'] == split_date]
# Separate features and target
X_train = train_data.drop(columns=['Customer ID', 'Consumption Month', 'Consumption'])
y_train = train_data['Consumption']
X_test = test_data.drop(columns=['Customer ID', 'Consumption Month', 'Consumption'])
y_test = test_data['Consumption']



/var/tmp/ipykernel_46543/3829726681.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_long.fillna(method='bfill', inplace=True)


In [48]:
data=X_test.head()

In [49]:
data

,lag_1,lag_2,lag_3,rolling_mean
15715,1313.997,1357.796,1710.277,1335.8965
15814,521.994,710.159,1034.286,616.0765
15914,1205.124,1165.265,1127.676,1185.1945
15960,190.483,234.024,748.332,212.2535
16060,-63.965,-66.433,-68.954,-65.1990


In [60]:
instances = data.to_dict(orient='records')

In [61]:
endpoint.predict(instances=instances).predictions

PermissionDenied: 403 Permission denied on resource project {119551364473}. [links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
, reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/{119551364473}"
}
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
]

In [51]:
my_model=aiplatform.Model("projects/119551364473/locations/us-central1/models/6444916049069473792")


In [53]:
my_model.predict(data)

AttributeError: 'Model' object has no attribute 'predict'